# 老生常談
* 聽別人的不如自己寫好。寫錯了，也沒關係，至少老師知道你哪裡錯。
* 一個不錯的狀況就是，湊出正確的答案，但不是好的寫法。之後看別人怎麼寫就會寫了。
* 寫作業的時候，順便寫一寫你是怎麼想的，有助於老師了解你的想法。

#1. 組裝的快樂
* 之前的Youbike範例展示了如何讀取Youbike資料。但今天我希望知道的訊息是，一天24小時內，或者一週內，腳踏車使用率的起伏，以嘗試解答，在這個偉大的台北市，什麼時候最多人用車？是否因為週一到週日而有所變化？而車輛變化最大的地方是哪裡？哪裡最有可能找不到停車站停車（這是某位碩士生的研究題目）？
* 為了達到這個目的，我需要獲取每個時間點每個腳踏車站的腳踏車數量，而不能只是想抓就抓，那我要如何讓他定時的抓呢？
* 你可以查詢run python scheduler/schedule，其中一個結果是http://resources.arcgis.com/en/help/main/10.1/index.html#//002100000038000000
，但這不是我們要的。
* 此時你再查詢「run python repeatedly」，你會找到這個網頁http://stackoverflow.com/questions/474528/what-is-the-best-way-to-repeatedly-execute-a-function-every-x-seconds-in-python
* 上面的說明指示，你可以import sched或time來讓你的程式「睡覺幾秒鐘」

## using time library
* Youbike的資料說明中有提示，他每五分鐘會update一次。
* 下面給你兩個程式，第一個部分是抓取Youbike的json資料，第二個部分是展示time.sleep()的功能，sleep()中所填的秒數，代表你每幾秒鐘要執行一次。而使用for-each來控制，你要抓幾次。
* 你的任務是，把兩個程式組裝起來，每五分鐘抓一次json資料，一共抓3個。

In [1]:
import urllib2
import json
import time

## time() practices

In [4]:
t = time.time()
print t # time in floating point format
now_str = time.strftime('%Y-%m-%d %H%M', time.localtime(t)) # convert time to string
print now_str

1426749480.87
2015-03-19 1518


* 在你目前這個ipynb的資料夾裡面建立一個新的folder叫做__youbike_data__，這樣比較好統一管理你的資料。
* 下面的程式碼可用來取得一次的腳踏車資料，並且讀取現在的時間，來作為檔名。

In [17]:
t = time.time() # Read current time
now_str = time.strftime('%Y%m%d%H%M', time.localtime(t)) # Format time
response = urllib2.urlopen('http://opendata.dot.taipei.gov.tw/opendata/gwjs_cityhall.json')
data = json.load(response) # load json
f = open('youbike_data/%s.json'%(now_str), 'wb') # Open an empty file
json.dump(data, f) # Save json data into the file
f.close() # close the file
print "%s.json"%(now_str)

201503191158.json


* 下面的程式碼最主要的功能是讓程式的執行休息（sleep）一段時間（以秒計算）
* 注意，為了要展示停止一秒的功能，所以下面的時間格式有考慮到seconds（'%Y%m%d%H%M%S'），但上面的例子的時間格式只考慮到minutes（'%Y%m%d%H%M'）。測試時，你加上秒數會比較好測試，但是，就作業要求而言，建議取到minutes，也就是採用上面的例子的時間格式。

In [8]:
for i in range(10):
    t = time.time()
    now_str = time.strftime('%Y%m%d%H%M%S', time.localtime(t))
    print now_str
    time.sleep(1)

NameError: name 'time' is not defined

#2.  發展
* 恭喜你完成上面的練習，現在我不想要存成三個檔案，我希望用一個dictionary把所有資料存下來。
* 例如說，每次傳回來的那份資料叫做data，我現在要開一個dictionary名字叫做__ydict = {}__，我希望裡面儲存的pair，key是你抓取的時間、value是你抓取的資料。
* 你的任務是，由上面兩個範例再組裝出另一個程式，這次要抓完三次資料後，每次抓的資料都把它放進__ydict__裡頭，最後才把它儲存為一個檔案__ydict.json__。

In [ ]:
ydict = {}
for i in range(10):
    t = time.time()
    now_str = time.strftime('%Y%m%d%H%M', time.localtime(t))
    response = urllib2.urlopen('http://opendata.dot.taipei.gov.tw/opendata/gwjs_cityhall.json')
    data = json.load(response) # load json
    ydict[now_str] = data
    time.sleep(3)
f = open('youbike_data/ydict.json', 'wb') # Open an empty file
json.dump(ydict, f) # Save json data into the file
f.close()

#3. YMD Validation
* 現在你希望也要能判斷閏年的2月有29天。
* 閏年的計算方式如該連結。http://email.hlbh.hlc.edu.tw/~teacher9/2basic/decision-tree/A24.htm
* 若輸入20150229要能夠出現警示錯誤訊息，若輸入20160229則會成功驗證。

In [7]:
a = input("please input a number: ")
suc = True
b=28
year = a/10000
if year%4==0 or (year%100==0 and year/100%4==0) or year%1000==0:
    b=29
mdict = {1:31, 2:b, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
if not 10000000 <= a < 100000000:
    print "ERROR: %d (NOT TYPYCAL DATE FORMAT)"%(a)
else:
    if not 10000000 < a < 30000000:
        print "ERROR: YEAR %d (SHOULD BE 2001<=Year<=2015)"%(a/10000)
        suc = False
    if not 1 <= a%10000/100 <= 12:
        print "ERROR: MONTH %d (SHOULD BE 1<=Month<=12)"%(a%10000/100)
        suc = False
    if not 1 <= a%100 <= mdict[a%10000/100]: # MODIFY THIS LINE
        print "ERROR: DATE %d (SHOULD BE 1<=Day<=%d)"%(a%100, mdict[a%10000/100]) # MODIFY THIS LINE
        suc = False
    if suc: 
        print "VALIDATED: YMD = %d/%d/%d"%(a/10000, a%10000/100, a%100)

please input a number: 20150229
ERROR: DATE 29 (SHOULD BE 1<=Day<=28)
